In [1]:
!git clone https://github.com/bscs16033/InsightFace_with_RetinaFace.git

Cloning into 'InsightFace_with_RetinaFace'...
remote: Enumerating objects: 12356, done.
remote: Counting objects: 100% (12356/12356), done.
remote: Compressing objects: 100% (11040/11040), done.
remote: Total 12356 (delta 1117), reused 12356 (delta 1117), pack-reused 0
Receiving objects: 100% (12356/12356), 492.03 MiB | 41.66 MiB/s, done.
Resolving deltas: 100% (1117/1117), done.
Checking out files: 100% (11447/11447), done.


In [39]:
cd /content/InsightFace_with_RetinaFace

/content/InsightFace_with_RetinaFace


In [40]:
cd insightface/models

/content/InsightFace_with_RetinaFace/insightface/models


## Downloading the InsightFace Pre-trained Model
The next step is to download the pre-trained InsightFace model, whose link is given in the original [InsightFace repo](https://github.com/deepinsight/insightface). I am donwloading ResNet-100, there are other options available as well. To download a different model, visit this [link](https://github.com/deepinsight/insightface/wiki/Model-Zoo).

In [9]:
!wget https://www.dropbox.com/s/tj96fsm6t6rq8ye/model-r100-arcface-ms1m-refine-v2.zip

--2020-02-05 15:17:07--  https://www.dropbox.com/s/tj96fsm6t6rq8ye/model-r100-arcface-ms1m-refine-v2.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tj96fsm6t6rq8ye/model-r100-arcface-ms1m-refine-v2.zip [following]
--2020-02-05 15:17:07--  https://www.dropbox.com/s/raw/tj96fsm6t6rq8ye/model-r100-arcface-ms1m-refine-v2.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc9494e09fe4136c2fbfb7471fa.dl.dropboxusercontent.com/cd/0/inline/AxhoOc0CzJ8RfJ0SMfSYISkJeMDDZS2ztCgr9ZLc7NbDMFySaj9dpeZyIe0_47kvFn5fIlUejJV-seN-Rdc2E_X_VM9D0eDkjP_-nCOZv0Bu6Q/file# [following]
--2020-02-05 15:17:08--  https://ucc9494e09fe4136c2fbfb7471fa.dl.dropboxusercontent.com/cd/0/inline/AxhoOc0CzJ8RfJ0SMfSYISkJeMDDZS2ztCgr9ZLc7NbDMFySaj9dpeZyIe0_

In [10]:
# getting the resnet-100 pretrained model acquired from original insightface repo
!unzip "model-r100-arcface-ms1m-refine-v2.zip"

Archive:  model-r100-arcface-ms1m-refine-v2.zip
   creating: model-r100-ii/
  inflating: model-r100-ii/log       
  inflating: model-r100-ii/model-0000.params  
  inflating: model-r100-ii/model-symbol.json  


In [11]:
# to run mxnet with cuda
!pip install mxnet-cu100

     |████████████████████████████████| 540.1MB 32kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [41]:
cd ../../src

/content/InsightFace_with_RetinaFace/src


## Custom Dataset Training
The next step is to train the classifier. If you want to add your own training data, add it in the directory `InsightFace_with_RetinaFace/datasets/train`

In [42]:
#generate face embeddings
!python faces_embedding.py --dataset="../datasets/train" --embeddings="outputs/embeddings.pickle"  --model="../insightface/models/model-r100-ii/model,0"

[INFO] quantifying faces...
loading ../insightface/models/model-r100-ii/model 0
[16:22:56] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v1.2.0. Attempting to upgrade...
[16:22:56] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
[16:22:57] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:22:57] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
[16:22:57] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:22:57] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
[16:22:57] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:22:57] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
[16:22:57] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[16:22:57] src/nnvm/legac

## Training Classifier
The classifier used to classify the feature embeddings is **Softmax**.

`my_model.h5` is the keras model that will be created after the training.

`le.pickle` is the labels corresponding to the embeddings data.

In [43]:
!pip install scikit-learn==0.20.1

     |████████████████████████████████| 5.4MB 2.7MB/s 
  Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [44]:
!python train_softmax.py --embeddings="outputs/embeddings.pickle" --model="outputs/my_model.h5" --le="outputs/le.pickle"

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `ra

## RetinaFace Integration
At this point, the **InsightFace** recognizer has been trained with the training data. Now, we will Integrate **RetinaFace** in this code. 


First, drag **retinaface** folder from `InsightFace_with_RetinaFace/insightface` to `InsightFace_with_RetinaFace`

In [45]:
cd ../retinaface/rcnn

/content/InsightFace_with_RetinaFace/retinaface/rcnn


In [16]:
!sudo pip install cython scikit-image easydict

In [46]:
cd ../

/content/InsightFace_with_RetinaFace/retinaface


In [18]:
!make

cd rcnn/cython/; python setup.py build_ext --inplace; rm -rf build; cd ../../
running build_ext
cythoning bbox.pyx to bbox.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/InsightFace_with_RetinaFace/retinaface/rcnn/cython/bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
cythoning anchors.pyx to anchors.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/InsightFace_with_RetinaFace/retinaface/rcnn/cython/anchors.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
cythoning cpu_nms.pyx to cpu_nms.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later

# Create New Folder
Create a folder named **pretrained_model** in `InsightFace_with_RetinaFace/retinaface`.

In [47]:
cd pretrained_model/

/content/InsightFace_with_RetinaFace/retinaface/pretrained_model


## Downloading the RetinaFace Pre-trained Model
The next step is to download the RetinaFace pre-trained model, whose link is given in the original [RetinaFace repo](https://github.com/deepinsight/insightface/tree/master/RetinaFace). I am donwloading **ResNet-50**, there are other options available as well. To download a different model, visit the above link.


In [33]:
!wget https://www.dropbox.com/s/53ftnlarhyrpkg2/retinaface-R50.zip

--2020-02-05 15:59:33--  https://www.dropbox.com/s/53ftnlarhyrpkg2/retinaface-R50.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/53ftnlarhyrpkg2/retinaface-R50.zip [following]
--2020-02-05 15:59:33--  https://www.dropbox.com/s/raw/53ftnlarhyrpkg2/retinaface-R50.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca043cd362dfb042ad0106d6bdb.dl.dropboxusercontent.com/cd/0/inline/AxhX4eKyIQ3O6_W0lCSX3zYn3ZniJPHBvGbUDks7FnrZ5wHGtKTCiGKB3TOJYIJ3YSbOHyW8vt8wrqMoi-Z6ZJ8zcO-pBkuFRCuipg67C5haoQ/file# [following]
--2020-02-05 15:59:34--  https://uca043cd362dfb042ad0106d6bdb.dl.dropboxusercontent.com/cd/0/inline/AxhX4eKyIQ3O6_W0lCSX3zYn3ZniJPHBvGbUDks7FnrZ5wHGtKTCiGKB3TOJYIJ3YSbOHyW8vt8wrqMoi-Z6ZJ8zcO-pBkuFRCuipg67C5haoQ/file
R

In [34]:
!unzip 'retinaface-R50.zip'

Archive:  retinaface-R50.zip
  inflating: R50-0000.params         
  inflating: R50-symbol.json         


In [48]:
cd ../../src

/content/InsightFace_with_RetinaFace/src


In [37]:
# testing on image
!python recognizer_image.py --image-in="../datasets/test/002.jpg" --image-out="outputs/002_result.jpg" --model="../insightface/models/model-r100-ii/model,0"

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
[16:07:14] src/nnvm/legacy_json_u

## Face Recognition in a Video
Now, we will move onto performing recognition in a video.

In [49]:
# testing on videos
!python recognizer_video.py --video-in="../datasets/videos_input/OK_Cersei_Lannister.mp4" --video-out="../datasets/videos_output/OK_Cersei_Lannister_result.mp4" --model="../insightface/models/model-r100-ii/model,0"

Using TensorFlow backend.
[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
[16:37:52] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v1.3.0. Attempting to upgrade...
[16:37:52] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
means [0. 0. 0.]
use_landmarks True
sym size: 9
--------args start---------
Namespace(det=0, embeddings='outputs/embeddings.pickle', flip=0, ga_model='', gpu=0, image_size='112,112', le='outputs/le.pickle', model='../insightface/models/model-r100-ii/model,0', mymodel='outputs/my_model.h5', threshold=1.24, video_in='../datasets/videos_input/OK_Cersei_Lannister.mp4', video_out='../datasets/videos_output/OK_Cersei_Lannister_result.mp4')
--------args end-----------
loading ../insightface/models/model-r100-ii/m